In [1]:
# Author: Yecheng Huang
# Updates by: Corrina Calanoc
import pandas as pd
import os

# Pull data on depression rates in each census tract in DC 

**Instructions:**
* Download [this file from Google Drive](https://drive.google.com/file/d/1Bx_6r8oAqIJ1sgLFo9n-BvN4Ql-qoSAd/view?usp=share_link) and save in a folder called 'raw_data' that is one folder above the current working directory
  * data comes from the following link: [only contains DC area](https://www.dchealthmatters.org/indicators/index/view?indicatorId=2867&localeTypeId=4&periodId=244&comparisonId=6863)
    * for more data, visit https://www.cdc.gov/places/

In [4]:
# create file structure path so that code knows where to pull and save data
data_folder = os.path.join(os.getcwd(), "data")
df = pd.read_csv(os.path.join(data_folder, 'depression_tracts_indicator_data_download_20221128.csv'))
print(len(df))
df.Location = df.Location.astype(str)

178


In [5]:
df.describe()

,Indicator Rate Value,Rate Lower Confidence Interval,Rate Upper Confidence Interval,Indicator Count Value,Indicator Count Value Units,Count Lower Confidence Interval,Count Upper Confidence Interval,Period of Measure,Technical Note,Breakout Title,...,Breakout Rate Value,Breakout Rate Value Units,Breakout Rate Lower Confidence Interval,Breakout Rate Upper Confidence Interval,Breakout Count Value,Breakout Count Value Units,Breakout Count Lower Confidence Interval,Breakout Count Upper Confidence Interval,Breakout Unstable,Breakout Footer
count,178.000000,178.000000,178.000000,0.0,0.0,0.0,0.0,178.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
mean,20.562921,19.318539,21.952809,NaN,NaN,NaN,NaN,2019.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
std,2.210932,1.971619,2.496012,NaN,NaN,NaN,NaN,0.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
min,15.000000,14.100000,16.200000,NaN,NaN,NaN,NaN,2019.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
25%,19.100000,18.025000,20.300000,NaN,NaN,NaN,NaN,2019.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
50%,20.450000,19.300000,21.700000,NaN,NaN,NaN,NaN,2019.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
75%,21.600000,20.400000,23.200000,NaN,NaN,NaN,NaN,2019.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
max,31.000000,28.600000,33.400000,NaN,NaN,NaN,NaN,2019.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [6]:
df.head(3)

,Indicator Name,What Is This Indicator,Location Type,Location,Indicator Rate Value,Indicator Rate Value Units,Rate Lower Confidence Interval,Rate Upper Confidence Interval,Indicator Count Value,Indicator Count Value Units,...,Breakout Rate Value,Breakout Rate Value Units,Breakout Rate Lower Confidence Interval,Breakout Rate Upper Confidence Interval,Breakout Count Value,Breakout Count Value Units,Breakout Count Lower Confidence Interval,Breakout Count Upper Confidence Interval,Breakout Unstable,Breakout Footer
0,Adults Ever Diagnosed with Depression,This indicator shows the percentage of adults ...,Census Tract,11001000100,21.2,percent,20.0,22.5,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Adults Ever Diagnosed with Depression,This indicator shows the percentage of adults ...,Census Tract,11001000201,29.6,percent,26.8,32.3,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Adults Ever Diagnosed with Depression,This indicator shows the percentage of adults ...,Census Tract,11001000202,24.5,percent,22.4,26.9,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [7]:
df['Location'].apply(lambda x: len(x)).unique()

array([11])

# Do this only if the data file downloaded only comes as granual as zip codes
### (Skip if the data file is already for each census tract)

In [8]:
df_1 = df[['Indicator Rate Value','Location']]
# download the mapping of zip codes to census tracts
zcta = pd.read_csv('https://www2.census.gov/geo/docs/maps-data/data/rel2020/zcta520/tab20_zcta520_tract20_natl.txt',
                   sep='|')
zcta.columns = zcta.columns.str.lower()
zcta['geoid_tract_20'] = zcta['geoid_tract_20'].astype(str)
#extract zipcode
zcta = zcta.dropna(subset=['geoid_zcta5_20'])
zcta['Location'] = zcta['namelsad_zcta5_20'].apply(lambda x:x[-5:])
#Join: transform zip to tract
df_2 = df_1.merge(zcta[['geoid_tract_20','Location']],on='Location')
df_2 = df_2.rename(columns={'Indicator Rate Value':'Depression Rate (Adult)'})
df_2.to_csv(os.path.join(data_folder, '2019_Depressionrate_tract.csv'),index=False)

# Join with CRE Equity Data
**Instructions**: Download [this file](https://drive.google.com/file/d/1m_4iSNWri-zay_ltKeTS4ndoosMi7T8r/view?usp=share_link) and then save in the 'raw_data' folder created above. 
* Data source: https://www.census.gov/programs-surveys/community-resilience-estimates/data/supplement.html

In [9]:
cre_equity = pd.read_csv(os.path.join(data_folder, 'CRE_Equity_Tract_19.csv'), encoding='latin1')

In [10]:
cre_equity.head(2)

,GEO_ID,NAME,POPUNI,PRED0_E,PRED0_M,PRED0_PE,PRED0_PM,PRED0_PF,PRED12_E,PRED12_M,...,Broadband_PF,No_Veh_PE,No_Veh_PM,No_Veh_PF,HO_Vac_PE,HO_Vac_PM,HO_Vac_PF,Rent_Vac_PE,Rent_Vac_PM,Rent_Vac_PF
0,1400000US01001020100,"Census Tract 201, Autauga County, Alabama",1924,720,285,37.42,14.82,0,644,302,...,0,1.0,1.5,-1,0.0,5.8,0,0.0,17.5,0
1,1400000US01001020200,"Census Tract 202, Autauga County, Alabama",2173,791,327,36.40,15.05,0,917,335,...,-1,14.2,7.9,0,0.0,7.0,0,15.2,11.2,0


In [11]:
# get the last 11 characters of the GEO_ID which contains the census tract
cre_equity['location_id'] = cre_equity['GEO_ID'].apply(lambda x: x[-11:]).astype(str)

In [12]:
# make sure that these census tract IDs are unique
print(len(cre_equity['location_id']))
print(cre_equity['location_id'].nunique())

73056
73056


In [13]:
# join the depression file and the census tract file
joined_file = df.merge(cre_equity, left_on = 'Location', right_on = 'location_id', how = 'outer')

In [14]:
# see how many rows did not get a match
print(joined_file['Location'].isna().sum()) # number of rows from the CRE file that did not match the depression file
print(joined_file['location_id'].isna().sum()) # number of rows from the depression file that did not match the CRE file

matched_rows = joined_file[(~joined_file['location_id'].isna() & ~joined_file['Location'].isna())].copy(deep=True)
print(len(matched_rows)) # number of census tracked that matched



72878
0
178


In [15]:
# write file and then add / update to this Google Drive folder: https://drive.google.com/drive/folders/19RfOfSc8TWcXz4FyNEEg8ixJrkYcYJ0W
matched_rows.to_csv(os.path.join(data_folder, 'joined_depression_cre.csv'), index=False)